In [3]:
import ast
import numpy as np
import os
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [74]:
file_list = os.listdir(".")
fns = []
for fn in sorted(file_list):
    if ".dump" in fn:
        fns.append(fn)

For creating sum-wise dataframe:

In [72]:
st = ""
for fn in fns:
    with open(fn, 'r') as f:
        st += f.read()

data = st.split("\n")

for i in range(len(data)):
    if len(data[i]) == 0:
        del data[i]

for i in range(len(data)):
    if len(data[i]) > 50:
        ts = data[i].split(",")[0]
        top10 = np.array(ast.literal_eval(",".join(data[i].split(",")[1:])))
        price, vol = top10[:20], top10[20:]
        vol_buy3, vol_buy5, vol_buy10 = vol[:10][-3:], vol[:10][-5:], vol[:10]
        vol_sell3, vol_sell5, vol_sell10 = vol[10:][:3], vol[10:][:5], vol[10:]
        
        st = "{}, {}, {}, {}, {}, {}, {}".format(ts, vol_buy3.sum(), vol_sell3.sum(), vol_buy5.sum(), vol_sell5.sum(), vol_buy10.sum(), vol_sell10.sum())
        data[i] = st.split(", ")
        
    else:
        data[i] = data[i].split(", ")

# Adding LTP:
for i in range(len(data)):
    if ((len(data[i]) == 7) & (len(data[i-1]) == 4)):
        data[i].append(data[i-1][-1])
        
# Creating a dataframe with following features:

"""
1. ts
2. buy3
3. sell3
4. buy5
5. sell5
6. buy10
7. sell10
8. ltp
"""
df = []
for d in data:
    if len(d) in [7,8]:
        df.append(d)

cols = ['ts',
'buy3',
'sell3',
'buy5',
'sell5',
'buy10',
"sell10",
'ltp']

df = pd.DataFrame(data = df, columns = cols)

# A preview of data:
df.head(40)

# There are `nan` values for top entries in **ltp** column. For that, we will drop the top 12 entries of **ltp** column.

df = df.iloc[11:,:]
df.fillna(method = 'ffill', inplace = True)
df.to_csv("first_sum_{}h.csv".format(len(fns)), index = False)

For creating product-wise dataframe:

In [75]:
st = ""
for fn in fns:
    with open(fn, 'r') as f:
        st += f.read()


data = st.split("\n")

for i in range(len(data)):
    if len(data[i]) == 0:
        del data[i]

for i in range(len(data)):
    if len(data[i]) > 50:
        ts = data[i].split(",")[0]
        top10 = np.array(ast.literal_eval(",".join(data[i].split(",")[1:])))
        price, vol = sorted(top10[:20]), top10[20:]
        vol_buy3, vol_buy5, vol_buy10 = np.array(vol[:10][-3:]), np.array(vol[:10][-5:]), np.array(vol[:10])
        vol_sell3, vol_sell5, vol_sell10 = np.array(vol[10:][:3]), np.array(vol[10:][:5]), np.array(vol[10:])
        price_buy3, price_buy5, price_buy10 = np.array(sorted(price[:10], reverse = True)[:3]), np.array(sorted(price[:10], reverse = True)[:5]), np.array(sorted(price[:10], reverse = True))
        price_sell3, price_sell5, price_sell10 = np.array(price[10:][:3]), np.array(price[10:][:5]), np.array(price[10:])
#         vol_buy3, vol_buy5, vol_buy10 = vol[:10][-3:], vol[:10][-5:], vol[:10]
#         vol_sell3, vol_sell5, vol_sell10 = vol[10:][:3], vol[10:][:5], vol[10:]
        
        st = "{}, {}, {}, {}, {}, {}, {}".format(ts, np.sum(price_buy3*vol_buy3), np.sum(price_sell3*vol_sell3), np.sum(price_buy5*vol_buy5), np.sum(price_sell5*vol_sell5), np.sum(price_buy10*vol_buy10), np.sum(price_sell10*vol_sell10))
        data[i] = st.split(", ")
        
    else:
        data[i] = data[i].split(", ")
        
# Adding LTP

for i in range(len(data)):
    if ((len(data[i]) == 7) & (len(data[i-1]) == 4)):
        data[i].append(data[i-1][-1])

# Creating dataframe

df = []
for d in data:
    if len(d) in [7,8]:
        df.append(d)

cols = ['ts',
'buy3',
'sell3',
'buy5',
'sell5',
'buy10',
"sell10",
'ltp']

df = pd.DataFrame(data = df, columns = cols)

# A preview of data:

df.head()

# There are `nan` values for top entries in **ltp** column. For that, we will drop the top 12 entries of **ltp** column.
df = df.iloc[11:,:]
df.fillna(method = 'ffill', inplace = True)
df.to_csv("first_product_{}h.csv".format(len(fns)), index = False)

In [33]:
df_sum, df_product = pd.read_csv("first_sum_{}h.csv".format(len(fns))), pd.read_csv("first_product_{}h.csv".format(len(fns)))
# df_sum["voi3"], df_sum["voi5"], df_sum["voi10"] = df_sum["buy3"] - df_sum["sell3"], df_sum["buy5"] - df_sum["sell5"], df_sum["buy10"] - df_sum["sell10"]
# df_sum["oir3"], df_sum["oir5"], df_sum["oir10"] = df_sum["buy3"]/df_sum["sell3"], df_sum["buy5"]/df_sum["sell5"], df_sum["buy10"]/df_sum["sell10"]

In [34]:
df_sum.shape

(1199591, 8)

For creating new columns:

* Order Imbalance Ratio(oir) for top 3, 5, and 10 orders from both sides;

* Volume Order Imbalance(voi) for top 3, 5, and 10 orders from both sides.

In [37]:
df_sum["oir3"], df_sum["oir5"], df_sum["oir10"] = df_sum["buy3"]/df_sum["sell3"], df_sum["buy5"]/df_sum["sell5"], df_sum["buy10"]/df_sum["sell10"]

In [ ]:
df = df[["ts", "buy3"]]

In [28]:
df_sum.head()

,ts,buy3,sell3,buy5,sell5,buy10,sell10,ltp
0,2018-06-25 09:29:01.860959,499188.0,4402636.0,1241052.0,4458756.0,2126950.0,4776818.0,6152.0
1,2018-06-25 09:29:01.861617,486880.0,4402636.0,1228744.0,4458756.0,2114642.0,4776818.0,6152.0
2,2018-06-25 09:29:01.862634,486880.0,4400636.0,1228744.0,4456756.0,2114642.0,4774818.0,6152.0
3,2018-06-25 09:29:01.863324,486880.0,4400948.0,1228744.0,4457068.0,2114642.0,4775130.0,6152.0
4,2018-06-25 09:29:01.863989,486880.0,4400996.0,1228744.0,4457116.0,2114642.0,4775178.0,6152.0


In [30]:
df_sum.head()

,ts,buy3,sell3,buy5,sell5,buy10,sell10,ltp,oir3,oir5,oir10
0,2018-06-25 09:29:01.860959,499188.0,4402636.0,1241052.0,4458756.0,2126950.0,4776818.0,6152.0,0.113384,0.278340,0.445265
1,2018-06-25 09:29:01.861617,486880.0,4402636.0,1228744.0,4458756.0,2114642.0,4776818.0,6152.0,0.110588,0.275580,0.442688
2,2018-06-25 09:29:01.862634,486880.0,4400636.0,1228744.0,4456756.0,2114642.0,4774818.0,6152.0,0.110639,0.275704,0.442874
3,2018-06-25 09:29:01.863324,486880.0,4400948.0,1228744.0,4457068.0,2114642.0,4775130.0,6152.0,0.110631,0.275684,0.442845
4,2018-06-25 09:29:01.863989,486880.0,4400996.0,1228744.0,4457116.0,2114642.0,4775178.0,6152.0,0.110630,0.275681,0.442840


New feature which represents change in ltp

In [11]:
df_sum['ltps'] = df_sum['ltp'].shift(1)

In [13]:
df_sum['ltp_ch'] = df_sum['ltp'] - df_sum['ltps']